This notebook serves as a sandbox for testing different hyperparameters for training the neural network. While this notebook can  be ran without a Nvidia GPU, the usage of one is reccomended as this notebook was written with using CUDA when running PyTorch in mind.

In [2]:
#Import neccessary libraries
import numpy as np
import pandas as pd
import torch as th

In [38]:
#Read data from dataset.csv and convert into numpy arrays
dataFrame = pd.read_csv("dataset.csv")
dataArray = dataFrame.to_numpy()
labels = dataArray[:, 34].astype(str)
features = np.delete(dataArray, 34, 1)

#Convert strings in labels array into ints and set as type 'int64'
#Additionally, create a list containing the string values respective to the int representations for easy conversion later on
intLabels = np.empty(4424, dtype = 'int64')
for i in range(4424):
    if(labels[i] == "Dropout"):
        intLabels[i] = 0
    elif(labels[i] == "Graduate"):
        intLabels[i] = 1
    elif(labels[i] == "Enrolled"):
        intLabels[i] = 2
labels = intLabels
labelsRefrence =["Dropout", "Graduate", "Enrolled"]

#Clean up features array by removing biased features, one-hot encoding, and standardization
features = np.delete(features, [0, 1, 2, 6, 7, 8, 9, 10, 11, 12, 15, 18], 1)

courseEncode = np.zeros((4424, 17), dtype = 'float32')
qualiEncode = np.zeros((4424, 17), dtype = 'float32')
for i in range(4424):
    course = features[i, 0]
    courseEncode[i, (course - 1)] = 1
    quali = features[i, 2]
    qualiEncode[i, (quali - 1)] = 1
hotFeatures = np.concatenate((courseEncode, qualiEncode), axis = 1)

boolFeatures = np.vstack((features[:, 1], features[:, 3], features[:, 4], features[:, 5])).astype('float32').T
unstdFeatures = np.delete(features, [0, 1, 2, 3, 4, 5], 1).astype('float32')
stdFeatures = np.empty((4424, 16), dtype = 'float32')
for i in range(16):
    mean = np.mean(unstdFeatures[:, i])
    std = np.std(unstdFeatures[:, i])
    for j in range(4424):
        stdFeatures[j, i] = (unstdFeatures[j, i] - mean) / std

features = np.concatenate((stdFeatures, boolFeatures, hotFeatures), axis = 1)

#Dataset class used for creating datasets out of the training and validation data
class Dataset(th.utils.data.Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        return self.features[idx, :], self.labels[idx]
    
#Function for creating dataloaders using the training and validation sets made with getSets
#The dataloader passes samples in batches of 79 as it is a factor of both 1106 and 3318
#which are the number of entries in the validation and training sets respectively
def buildLoaders(tFeatures, tLabels, vFeatures, vLabels):
    tDataset = Dataset(tFeatures, tLabels)
    vDataset = Dataset(vFeatures, vLabels)
    tLoader = th.utils.data.DataLoader(tDataset, batch_size = 79)
    vLoader = th.utils.data.DataLoader(vDataset, batch_size = 79,)
    return tLoader, vLoader

#NeuralNetwork class used for the building the model
#Neural Network sepcific hyperparameters are edited directly in this class
class NeuralNetwork(th.nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = th.nn.Flatten()
        self.linearReLUStack = th.nn.Sequential(
            th.nn.Linear(54, 30),
            th.nn.ReLU(),
            th.nn.Dropout(0.2),
            th.nn.Linear(30, 3)
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linearReLUStack(x)
        return logits
    
#Select the device to be used for training the model and send the model to it
#Uses GPU if a Nvdia GPU is detected and PyTorch was installed with the CUDA platform, uses CPU otherwise
device = "cuda" if th.cuda.is_available() else "cpu"
model = NeuralNetwork().to(device)

#Define a loss function and an optimizer for training the model
lossFunc = th.nn.CrossEntropyLoss()
optimizer = th.optim.SGD(model.parameters(), lr = 0.0001, momentum = 0.9, weight_decay = 0.001)

#Define training function
def train(tLoader, model, lossFunc, optimizer):
    size = len(tLoader.dataset)
    model.train()
    for batch, (X, y) in enumerate(tLoader):
        X, y = X.to(device), y.to(device)
        
        #Compute Prediction Error
        prediction = model(X)
        loss = lossFunc(prediction, y)
        
        #Backpropagate
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

#Define testing fuction
def test(vLoader, model, lossFunc):
    size = len(vLoader.dataset)
    numBatches = len(vLoader)
    model.eval()
    tLoss, correct = 0, 0
    with th.no_grad():
        for X, y in vLoader:
            X, y = X.to(device), y.to(device)
            prediction = model(X)
            tLoss += lossFunc(prediction, y).item()
            correct += (prediction.argmax(1) == y).type(th.float).sum().item()
    tLoss /= numBatches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {tLoss:>8f} \n")

#Create datasets for trianing data and validation data
pIndex = np.random.permutation(4424)
vNum = int(4424 * 0.2)
vIndex = pIndex[:vNum]
tIndex = pIndex[vNum:]
tFeatures = features[tIndex]
tLabels = labels[tIndex]
vFeatures = features[vIndex]
vLabels = labels[vIndex]

#Train the model for the given number of epochs and print out the training and testing error
epochs = 200
tLoader, vLoader = buildLoaders(tFeatures, tLabels, vFeatures, vLabels)
for i in range(epochs):
    print(f"Epoch {i+1}\n-------------------------------")
    train(tLoader, model, lossFunc, optimizer)
    test(vLoader, model, lossFunc)


Epoch 1
-------------------------------
loss: 1.117127  [   79/ 3540]
loss: 1.097359  [  158/ 3540]
loss: 1.101886  [  237/ 3540]
loss: 1.094858  [  316/ 3540]
loss: 1.095789  [  395/ 3540]
loss: 1.099563  [  474/ 3540]
loss: 1.094337  [  553/ 3540]
loss: 1.114785  [  632/ 3540]
loss: 1.082783  [  711/ 3540]
loss: 1.103864  [  790/ 3540]
loss: 1.089203  [  869/ 3540]
loss: 1.117481  [  948/ 3540]
loss: 1.111617  [ 1027/ 3540]
loss: 1.108325  [ 1106/ 3540]
loss: 1.091695  [ 1185/ 3540]
loss: 1.103143  [ 1264/ 3540]
loss: 1.096269  [ 1343/ 3540]
loss: 1.102209  [ 1422/ 3540]
loss: 1.092416  [ 1501/ 3540]
loss: 1.097554  [ 1580/ 3540]
loss: 1.110419  [ 1659/ 3540]
loss: 1.089133  [ 1738/ 3540]
loss: 1.099491  [ 1817/ 3540]
loss: 1.107995  [ 1896/ 3540]
loss: 1.108160  [ 1975/ 3540]
loss: 1.090955  [ 2054/ 3540]
loss: 1.089776  [ 2133/ 3540]
loss: 1.091208  [ 2212/ 3540]
loss: 1.103149  [ 2291/ 3540]
loss: 1.099142  [ 2370/ 3540]
loss: 1.106365  [ 2449/ 3540]
loss: 1.087532  [ 2528/ 3540]


loss: 1.053342  [  158/ 3540]
loss: 1.043178  [  237/ 3540]
loss: 1.042698  [  316/ 3540]
loss: 1.033436  [  395/ 3540]
loss: 1.028232  [  474/ 3540]
loss: 1.030787  [  553/ 3540]
loss: 1.069941  [  632/ 3540]
loss: 1.040079  [  711/ 3540]
loss: 1.049571  [  790/ 3540]
loss: 1.039552  [  869/ 3540]
loss: 1.061967  [  948/ 3540]
loss: 1.053646  [ 1027/ 3540]
loss: 1.058305  [ 1106/ 3540]
loss: 1.053638  [ 1185/ 3540]
loss: 1.065417  [ 1264/ 3540]
loss: 1.021034  [ 1343/ 3540]
loss: 1.033186  [ 1422/ 3540]
loss: 1.071887  [ 1501/ 3540]
loss: 1.049292  [ 1580/ 3540]
loss: 1.066300  [ 1659/ 3540]
loss: 1.020857  [ 1738/ 3540]
loss: 1.052144  [ 1817/ 3540]
loss: 1.059719  [ 1896/ 3540]
loss: 1.052012  [ 1975/ 3540]
loss: 1.051108  [ 2054/ 3540]
loss: 1.024469  [ 2133/ 3540]
loss: 1.047134  [ 2212/ 3540]
loss: 1.037630  [ 2291/ 3540]
loss: 1.032120  [ 2370/ 3540]
loss: 1.054631  [ 2449/ 3540]
loss: 1.005203  [ 2528/ 3540]
loss: 1.019267  [ 2607/ 3540]
loss: 1.051852  [ 2686/ 3540]
loss: 1.02

loss: 0.987294  [  474/ 3540]
loss: 0.993686  [  553/ 3540]
loss: 1.046859  [  632/ 3540]
loss: 0.999826  [  711/ 3540]
loss: 1.023224  [  790/ 3540]
loss: 1.000895  [  869/ 3540]
loss: 1.045844  [  948/ 3540]
loss: 1.020467  [ 1027/ 3540]
loss: 1.017724  [ 1106/ 3540]
loss: 1.024192  [ 1185/ 3540]
loss: 1.029701  [ 1264/ 3540]
loss: 0.997588  [ 1343/ 3540]
loss: 1.005967  [ 1422/ 3540]
loss: 1.061854  [ 1501/ 3540]
loss: 1.011110  [ 1580/ 3540]
loss: 1.036345  [ 1659/ 3540]
loss: 0.977846  [ 1738/ 3540]
loss: 1.013216  [ 1817/ 3540]
loss: 1.042388  [ 1896/ 3540]
loss: 1.017601  [ 1975/ 3540]
loss: 0.998642  [ 2054/ 3540]
loss: 0.993070  [ 2133/ 3540]
loss: 1.008678  [ 2212/ 3540]
loss: 1.011212  [ 2291/ 3540]
loss: 1.013957  [ 2370/ 3540]
loss: 1.020928  [ 2449/ 3540]
loss: 0.977986  [ 2528/ 3540]
loss: 0.992626  [ 2607/ 3540]
loss: 1.016339  [ 2686/ 3540]
loss: 0.976871  [ 2765/ 3540]
loss: 1.027197  [ 2844/ 3540]
loss: 1.006774  [ 2923/ 3540]
loss: 1.012392  [ 3002/ 3540]
loss: 1.00

loss: 0.944715  [ 2528/ 3540]
loss: 0.963789  [ 2607/ 3540]
loss: 0.992122  [ 2686/ 3540]
loss: 0.946379  [ 2765/ 3540]
loss: 1.013221  [ 2844/ 3540]
loss: 0.989834  [ 2923/ 3540]
loss: 0.981898  [ 3002/ 3540]
loss: 0.979478  [ 3081/ 3540]
loss: 0.977288  [ 3160/ 3540]
loss: 0.976139  [ 3239/ 3540]
loss: 0.987687  [ 3318/ 3540]
loss: 0.987963  [ 3397/ 3540]
loss: 1.013039  [ 3476/ 3540]
loss: 0.980980  [ 2880/ 3540]
Test Error: 
 Accuracy: 47.4%, Avg loss: 1.019357 

Epoch 24
-------------------------------
loss: 1.000492  [   79/ 3540]
loss: 1.023954  [  158/ 3540]
loss: 0.969838  [  237/ 3540]
loss: 0.967007  [  316/ 3540]
loss: 0.975903  [  395/ 3540]
loss: 0.954792  [  474/ 3540]
loss: 0.952894  [  553/ 3540]
loss: 1.021363  [  632/ 3540]
loss: 0.987702  [  711/ 3540]
loss: 0.998196  [  790/ 3540]
loss: 0.960578  [  869/ 3540]
loss: 1.027057  [  948/ 3540]
loss: 1.006696  [ 1027/ 3540]
loss: 0.988664  [ 1106/ 3540]
loss: 1.007658  [ 1185/ 3540]
loss: 1.018572  [ 1264/ 3540]
loss: 0

loss: 0.985028  [   79/ 3540]
loss: 1.005337  [  158/ 3540]
loss: 0.950243  [  237/ 3540]
loss: 0.961873  [  316/ 3540]
loss: 0.953278  [  395/ 3540]
loss: 0.918649  [  474/ 3540]
loss: 0.925350  [  553/ 3540]
loss: 1.012177  [  632/ 3540]
loss: 0.953821  [  711/ 3540]
loss: 0.965979  [  790/ 3540]
loss: 0.947763  [  869/ 3540]
loss: 1.011552  [  948/ 3540]
loss: 0.966301  [ 1027/ 3540]
loss: 0.979968  [ 1106/ 3540]
loss: 0.983175  [ 1185/ 3540]
loss: 1.009452  [ 1264/ 3540]
loss: 0.922537  [ 1343/ 3540]
loss: 0.935697  [ 1422/ 3540]
loss: 1.021898  [ 1501/ 3540]
loss: 0.974483  [ 1580/ 3540]
loss: 0.987302  [ 1659/ 3540]
loss: 0.906316  [ 1738/ 3540]
loss: 0.946150  [ 1817/ 3540]
loss: 0.999621  [ 1896/ 3540]
loss: 0.975748  [ 1975/ 3540]
loss: 0.958983  [ 2054/ 3540]
loss: 0.936485  [ 2133/ 3540]
loss: 0.982495  [ 2212/ 3540]
loss: 0.946177  [ 2291/ 3540]
loss: 0.930932  [ 2370/ 3540]
loss: 0.976957  [ 2449/ 3540]
loss: 0.898459  [ 2528/ 3540]
loss: 0.928651  [ 2607/ 3540]
loss: 0.97

...

loss: 0.811987  [ 1501/ 3540]
loss: 0.746090  [ 1580/ 3540]
loss: 0.789827  [ 1659/ 3540]
loss: 0.671341  [ 1738/ 3540]
loss: 0.619952  [ 1817/ 3540]
loss: 0.789775  [ 1896/ 3540]
loss: 0.711456  [ 1975/ 3540]
loss: 0.695628  [ 2054/ 3540]
loss: 0.678254  [ 2133/ 3540]
loss: 0.808235  [ 2212/ 3540]
loss: 0.657484  [ 2291/ 3540]
loss: 0.687895  [ 2370/ 3540]
loss: 0.756691  [ 2449/ 3540]
loss: 0.696692  [ 2528/ 3540]
loss: 0.667322  [ 2607/ 3540]
loss: 0.811254  [ 2686/ 3540]
loss: 0.663248  [ 2765/ 3540]
loss: 0.776767  [ 2844/ 3540]
loss: 0.712015  [ 2923/ 3540]
loss: 0.693507  [ 3002/ 3540]
loss: 0.661296  [ 3081/ 3540]
loss: 0.722164  [ 3160/ 3540]
loss: 0.761316  [ 3239/ 3540]
loss: 0.846031  [ 3318/ 3540]
loss: 0.742998  [ 3397/ 3540]
loss: 0.740548  [ 3476/ 3540]
loss: 0.713792  [ 2880/ 3540]
Test Error: 
 Accuracy: 68.2%, Avg loss: 0.781283 

Epoch 132
-------------------------------
loss: 0.698150  [   79/ 3540]
loss: 0.756051  [  158/ 3540]
loss: 0.687083  [  237/ 3540]
loss: 

loss: 0.741846  [ 2923/ 3540]
loss: 0.692660  [ 3002/ 3540]
loss: 0.648719  [ 3081/ 3540]
loss: 0.730035  [ 3160/ 3540]
loss: 0.735758  [ 3239/ 3540]
loss: 0.826719  [ 3318/ 3540]
loss: 0.714863  [ 3397/ 3540]
loss: 0.730702  [ 3476/ 3540]
loss: 0.679669  [ 2880/ 3540]
Test Error: 
 Accuracy: 68.3%, Avg loss: 0.772978 

Epoch 139
-------------------------------
loss: 0.720086  [   79/ 3540]
loss: 0.749918  [  158/ 3540]
loss: 0.676787  [  237/ 3540]
loss: 0.765127  [  316/ 3540]
loss: 0.728900  [  395/ 3540]
loss: 0.560240  [  474/ 3540]
loss: 0.595748  [  553/ 3540]
loss: 0.804091  [  632/ 3540]
loss: 0.688462  [  711/ 3540]
loss: 0.764681  [  790/ 3540]
loss: 0.608887  [  869/ 3540]
loss: 0.786111  [  948/ 3540]
loss: 0.643976  [ 1027/ 3540]
loss: 0.734970  [ 1106/ 3540]
loss: 0.738408  [ 1185/ 3540]
loss: 0.827072  [ 1264/ 3540]
loss: 0.678212  [ 1343/ 3540]
loss: 0.647197  [ 1422/ 3540]
loss: 0.799816  [ 1501/ 3540]
loss: 0.721460  [ 1580/ 3540]
loss: 0.761872  [ 1659/ 3540]
loss: 

Test Error: 
 Accuracy: 68.6%, Avg loss: 0.765012 

Epoch 146
-------------------------------
loss: 0.691436  [   79/ 3540]
loss: 0.756532  [  158/ 3540]
loss: 0.650774  [  237/ 3540]
loss: 0.777959  [  316/ 3540]
loss: 0.736212  [  395/ 3540]
loss: 0.590036  [  474/ 3540]
loss: 0.648786  [  553/ 3540]
loss: 0.783536  [  632/ 3540]
loss: 0.678943  [  711/ 3540]
loss: 0.717587  [  790/ 3540]
loss: 0.600002  [  869/ 3540]
loss: 0.779972  [  948/ 3540]
loss: 0.648185  [ 1027/ 3540]
loss: 0.733939  [ 1106/ 3540]
loss: 0.725743  [ 1185/ 3540]
loss: 0.808323  [ 1264/ 3540]
loss: 0.667153  [ 1343/ 3540]
loss: 0.657147  [ 1422/ 3540]
loss: 0.812422  [ 1501/ 3540]
loss: 0.747256  [ 1580/ 3540]
loss: 0.740198  [ 1659/ 3540]
loss: 0.638701  [ 1738/ 3540]
loss: 0.622298  [ 1817/ 3540]
loss: 0.806876  [ 1896/ 3540]
loss: 0.690292  [ 1975/ 3540]
loss: 0.678113  [ 2054/ 3540]
loss: 0.635470  [ 2133/ 3540]
loss: 0.805584  [ 2212/ 3540]
loss: 0.635882  [ 2291/ 3540]
loss: 0.671523  [ 2370/ 3540]
loss: 

loss: 0.634132  [ 2880/ 3540]
Test Error: 
 Accuracy: 68.7%, Avg loss: 0.757601 

Epoch 153
-------------------------------
loss: 0.693901  [   79/ 3540]
loss: 0.728864  [  158/ 3540]
loss: 0.655372  [  237/ 3540]
loss: 0.772956  [  316/ 3540]
loss: 0.757729  [  395/ 3540]
loss: 0.550274  [  474/ 3540]
loss: 0.633016  [  553/ 3540]
loss: 0.808674  [  632/ 3540]
loss: 0.663568  [  711/ 3540]
loss: 0.713053  [  790/ 3540]
loss: 0.608579  [  869/ 3540]
loss: 0.788517  [  948/ 3540]
loss: 0.632986  [ 1027/ 3540]
loss: 0.677774  [ 1106/ 3540]
loss: 0.729846  [ 1185/ 3540]
loss: 0.838795  [ 1264/ 3540]
loss: 0.670195  [ 1343/ 3540]
loss: 0.641827  [ 1422/ 3540]
loss: 0.793454  [ 1501/ 3540]
loss: 0.731201  [ 1580/ 3540]
loss: 0.770686  [ 1659/ 3540]
loss: 0.674915  [ 1738/ 3540]
loss: 0.590224  [ 1817/ 3540]
loss: 0.778481  [ 1896/ 3540]
loss: 0.679036  [ 1975/ 3540]
loss: 0.664235  [ 2054/ 3540]
loss: 0.611639  [ 2133/ 3540]
loss: 0.777326  [ 2212/ 3540]
loss: 0.652810  [ 2291/ 3540]
loss: 

KeyboardInterrupt: 